In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7194,2024-07-21,Eua Nba,21:30,Utah Jazz,Detroit Pistons,2.10,1.77,173.5,1.91,1.91,1.5,2.00,1.57,UNU8B3c6,0.476190,0.564972,0.523560,0.523560,0.041162,227.804,153.363122,0.673224,1.2,1.643168,1.369306,245.52,2.390,1.147105,0.479960,-25.0,219.772,68.883304,0.313431,1.2,1.643168,1.369306,150.30,2.494,0.675115,0.270696,-20.0,88,90,2.79,1.67,373.112,487.104,0.120592,0.000000,0.170339,-0.71,-0.142,-7.746479,0.379082,0.2,-0.179082,4.00,0.800,0.962500,0.281508,0.3,0.018492
7195,2024-07-21,Chile Lnb 2,19:30,Arabe Valparaiso,Municipal Chillan,1.93,1.79,128.5,1.83,1.87,0.0,0.00,0.00,p0UGJ9eb,0.518135,0.558659,0.546448,0.534759,0.076794,173.712,77.587239,0.446643,0.0,0.000000,NaN,159.82,2.764,1.100332,0.398094,-75.0,183.528,143.544068,0.782137,1.8,1.643168,0.912871,134.40,2.992,2.737219,0.914846,26.0,61,60,2.62,2.24,109.292,306.208,0.053223,0.015289,NaN,-5.00,-1.000,-0.930000,0.000000,0.0,0.000000,-0.31,-0.062,-12.741935,0.000000,0.0,0.000000
7196,2024-07-21,Colômbia Lbp,17:30,Toros del Valle,Titanes de Barranquilla,2.50,1.51,168.5,1.83,1.83,0.0,0.00,0.00,dj07ObWi,0.400000,0.662252,0.546448,0.546448,0.062252,228.636,46.473866,0.203266,1.2,1.643168,1.369306,225.00,2.586,0.650484,0.251541,-20.0,121.744,16.381193,0.134554,2.4,1.341641,0.559017,140.06,1.438,0.102811,0.071495,43.0,90,94,2.50,1.49,169.448,128.094,0.349145,0.000000,NaN,-0.17,-0.034,-44.117647,0.562417,0.6,0.037583,0.48,0.096,5.312500,0.671132,0.8,0.128868
7197,2024-07-21,Eua Nba,16:00,Minnesota Timberwolves,Orlando Magic,1.77,2.10,180.5,1.91,1.91,-2.5,1.95,2.40,pCQKGwIl,0.564972,0.476190,0.523560,0.523560,0.041162,154.340,29.946739,0.194031,1.8,1.643168,0.912871,197.16,1.634,0.274463,0.167970,5.0,177.006,43.462404,0.245542,1.2,1.643168,1.369306,143.00,1.812,0.456859,0.252130,12.0,93,100,2.12,1.43,147.308,198.166,0.120592,0.000000,0.146298,-3.54,-0.708,-1.087571,0.627014,0.4,-0.227014,-3.63,-0.726,-1.515152,0.534026,0.2,-0.334026
7198,2024-07-21,Eua Nba,18:00,Boston Celtics,Philadelphia 76ers,2.38,1.63,181.5,1.91,1.91,2.5,2.00,1.44,Uq6qOtsC,0.420168,0.613497,0.523560,0.523560,0.033665,157.860,45.987657,0.291319,1.8,1.643168,0.912871,128.70,1.604,0.296108,0.184606,21.0,182.504,41.494027,0.227360,1.8,1.643168,0.912871,207.36,1.866,0.468167,0.250893,26.0,90,96,1.43,2.16,164.576,223.838,0.264504,0.000000,0.230221,-0.95,-0.190,-7.263158,0.781703,0.7,-0.081703,3.37,0.674,0.934718,0.513140,0.7,0.186860
7199,2024-07-21,Eua Nba,19:30,Brooklyn Nets,Charlotte Hornets,2.18,1.73,175.5,1.91,1.91,1.5,2.00,1.56,0rwhFobs,0.458716,0.578035,0.523560,0.523560,0.036750,293.344,222.362812,0.758027,1.8,1.643168,0.912871,284.58,3.268,2.621788,0.802261,-19.0,268.388,285.855211,1.065082,2.4,1.341641,0.559017,146.16,2.622,2.181839,0.832128,39.0,102,84,2.79,1.74,193.310,406.930,0.162761,0.000000,0.174790,2.02,0.404,2.920792,0.459480,0.7,0.240520,10.04,2.008,0.363546,0.346136,0.5,0.153864
7200,2024-07-21,Eua Nba,20:00,San Antonio Spurs,Toronto Raptors,1.73,2.18,172.5,1.91,1.91,-3.5,2.00,2.75,WSnMK7zK,0.578035,0.458716,0.523560,0.523560,0.036750,166.282,20.007058,0.120320,2.4,1.341641,0.559017,134.40,1.872,0.358357,0.191430,26.0,407.094,567.413921,1.393815,1.2,1.643168,1.369306,184.69,4.280,5.334290,1.246329,-33.0,80,73,1.68,2.53,266.644,540.770,0.162761,0.000000,0.223297,1.99,0.398,1.834171,0.395949,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7198,18:00,Eua Nba,Boston Celtics,Philadelphia 76ers,2.38,Back Home
7200,20:00,Eua Nba,San Antonio Spurs,Toronto Raptors,1.73,Back Home
